# RNN

### 時系列データ
時間方向に並んだデータのこと

### RNNで扱われる時系列データ一覧

*   文書
*   音声データ
*   動画
*   株価
*   産業用機器の状態  
etc...

### 再帰型ニューラルネットワーク (Reccurent Neural Network)
入力と正解が時系列データになる  
中間層が再帰の構造を持ち、前後の時刻の中間層とつながる

### LSTM（Long Short Term Memory）
RNNの一種  
記憶セルの導入による、長期記憶保持  
ゲートと呼ばれる情報の流れを調整する仕組み  
ゲートにより、記憶セルの内容を忘れるか忘れないか判断しながら、必要な情報だけを次の時刻に引き継ぐことができる  
記憶セルは貯水池、各ゲートは、水門のようなイメージ

### MagentaのRNN音楽生成モデル

*   Melody RNN（単音のシンプルなメロディー作曲）
*   Drums RNN（ドラム演奏を作曲）
*   Improv RNN（コード進行に沿って、アドリブでメロディーの作曲）
*   Polyphony RNN（和音を繋いで進行する曲を生成）
*   Pianoroll RNN-NADE（ピアノロールを使用して現代音楽風の曲を生成）
*   Performance RNN（音の強弱のタイミングなどの微妙な表現まで変化する高度な曲の生成）






In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

In [ ]:
import magenta
import note_seq
from note_seq.protobuf import music_pb2

seed = music_pb2.NoteSequence()

seed.notes.add(pitch=80, start_time=0.0, end_time=0.4, velocity=80)
seed.notes.add(pitch=80, start_time=0.4, end_time=0.8, velocity=80)
seed.notes.add(pitch=87, start_time=0.8, end_time=1.2, velocity=80)
seed.notes.add(pitch=87, start_time=1.2, end_time=1.6, velocity=80)
seed.notes.add(pitch=89, start_time=1.6, end_time=2.0, velocity=80)
seed.notes.add(pitch=89, start_time=2.0, end_time=2.4, velocity=80)
seed.notes.add(pitch=87, start_time=2.4, end_time=3.2, velocity=80)

seed.total_time = 3.2
seed.tempos.add(qpm=75)

note_seq.plot_sequence(seed)
note_seq.play_sequence(seed, synth=note_seq.fluidsynth)

In [ ]:
from magenta.models.polyphony_rnn import polyphony_sequence_generator
from magenta.models.shared import sequence_generator_bundle

# model initialize
note_seq.notebook_utils.download_bundle("polyphony_rnn.mag", "/models/") # bundle(.magファイル)をダウンロード
bundle = sequence_generator_bundle.read_bundle_file("/models/polyphony_rnn.mag")
generator_map = polyphony_sequence_generator.get_generator_map()
polyphony_rnn = generator_map["polyphony"](checkpoint=None, bundle=bundle)
polyphony_rnn.initialize()

In [ ]:
from note_seq.protobuf import generator_pb2

total_time = 180
temperature = 1.0

base_end_time = max(note.end_time for note in seed.notes)

generator_options = generator_pb2.GeneratorOptions()
generator_options.args["temperature"].float_value = temperature
generator_options.generate_sections.add(
    start_time=base_end_time,
    end_time=total_time)

gen_seq = polyphony_rnn.generate(seed, generator_options)
note_seq.plot_sequence(gen_seq)
note_seq.play_sequence(gen_seq, synth=note_seq.fluidsynth)